# Import libraries

In [1]:
import glob
import os
from pathlib import Path

import pandas as pd
pd.options.display.max_rows = 1000

os.environ["SCHRODINGER_PATH"] = "/datos/software/schrodinger/schrodinger2020-3"
os.environ["SCHRODINGER_PATH"] = "/mnt/c/Program Files/Schrodinger2021-4"
import polypharm as ppm

# Variable settings

In [2]:
# Tesseract
# WORKING_FOLDER = "/mnt/c/Users/kcoru/Dropbox/wendy/lorena/scripts_tmp/02_notebooks"
# SCHRODINGER_PATH = "/mnt/c/software/maestro"
# Stark
WORKDIR = Path("/home/mbedoya/Dropbox/wendy/lorena/scripts_tmp/02_notebooks")
WORKDIR = Path("/home/fadasme/Downloads/polypharm_workdir")

LIGAND_DIR = WORKDIR / "01_ligands"
PROTEIN_DIR = WORKDIR / "02_proteins"
IFD_DIR = WORKDIR / "03_ifd"
MMGBSA_DIR = WORKDIR / "04_mmgbsa"
ANALYSIS_DIR = WORKDIR / "05_analysis"

CONTACT_CUTOFF = 5.0
BS_RESIDS = {
    "kv1.5_open_cc": "A:436,A:439,A:440,A:443,A:502,A:510,B:436,B:439,B:440,B:443,B:502,B:510",
    "nav1.5_6lqa": "B:1462,B:1466,B:1760,B:1767",
    "task1_6rv3": "A:126,A:171,A:194,A:198,A:199,A:234,A:235,A:236,A:238,A:239,A:240,B:93,B:111,B:114,B:118",
    "kv1.5_open_cc_sp": "A:436,A:439,A:440,A:443,A:502,A:510,B:436,B:439,B:440,B:443,B:502,B:510",
}
RANK_CRITERIA = [
    ppm.RankingCriterion.NORMALIZED_CONTACTS,
    ppm.RankingCriterion.TOTAL_SCORE,
]

# Running IFD calculations

In [ ]:
prot_files = glob.glob(os.path.join(PROTEIN_DIR, "*.mae*"))
lig_files = glob.glob(os.path.join(LIGAND_DIR, "*.mae*"))
ppm.run_ifd_cross(
    prot_files=prot_files,
    lig_files=lig_files,
    bs_residues=BS_RESIDS,
    workdir=IFD_DIR,
    glide_cpus=2,
    prime_cpus=2,
    tasks=10,
)

# Running MMGBSA calculations

In [ ]:
ppm.run_mmgbsa_cross(
    glob.glob(os.path.join(IFD_DIR, "**", "*", "*-out.maegz")),
    workdir=MMGBSA_DIR,
    cpus=2,
    tasks=10,
)

# Running analysis

In [5]:
import subprocess
try:
    results = ppm.report(MMGBSA_DIR, BS_RESIDS, CONTACT_CUTOFF, tasks=10)
except subprocess.CalledProcessError as ex:
    print(" ".join(ex.cmd))
    print(ex.stdout.decode())
results.head(10)

,PROTEIN,NAME,INDEX,IFDSCORE,DGBIND,INT,A:126,A:171,A:194,A:198,...,B:436,B:439,B:440,B:443,B:502,B:510,B:1462,B:1466,B:1760,B:1767
0,task1_6rv3,LA-2_prot,0,-1048.634491,-48.502970,11,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,task1_6rv3,LA-2_prot,1,-1048.538242,-54.210013,8,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,task1_6rv3,LA-2_prot,2,-1048.391644,-43.687662,10,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,task1_6rv3,LA-2_prot,3,-1047.833339,-50.133405,8,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,task1_6rv3,LA-2_prot,4,-1047.539312,-42.111163,11,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,task1_6rv3,LA-2_prot,5,-1047.383051,-45.858787,9,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,task1_6rv3,LA-2_prot,6,-1047.280469,-42.777236,11,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,task1_6rv3,LA-2_prot,7,-1047.046917,-49.475656,10,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,task1_6rv3,LA-2_prot,8,-1046.843575,-50.345990,9,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,task1_6rv3,LA-2_prot,9,-1046.621825,-45.145327,8,0.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ranked_results = ppm.rank_poses(results, RANK_CRITERIA)
ranked_results[ranked_results["NAME"] == "LA-4_prot"].dropna(axis=1).sort_values("RANK")

In [ ]:
cross_results = ppm.rank_poses_cross(results, RANK_CRITERIA)
cross_results.T